In [1]:
# 5-2
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [2]:
# 5-3
# =========================================================
# 目的：
#  - 日本語BERT（Whole Word Masking 版）を「MLM（Masked Language Modeling）」タスク用の
#    モデルとしてロードし、MacBook を含む環境で最適なデバイス（MPS/CUDA/CPU）に載せる。
#
# 理論メモ：
#  - MLM（Masked Language Modeling）は、入力トークンの一部を [MASK] に置換し、
#    その復元を学習する自己教師ありタスクである。BERTの事前学習の主要素。
#  - Whole Word Masking（WWM）は「サブワード単位ではなく“語”単位でまとめてマスクする」
#    戦略。語彙断片ではなく語レベルの完形復元圧力を与えるため、語彙的一貫性の向上が期待される。
#  - 推論（補完）時は [MASK] の位置の語彙分布（logits）から上位候補を選ぶ。
#    学習時は [MASK] 位置以外のロスを無視（ignore_index=-100）するのが一般的（DataCollatorが自動対応可）。
#  - 日本語BERT（BertJapaneseTokenizer）は「形態素解析→WordPiece」の二段で分割する。
#    分割は辞書・語彙・バージョンに依存するため、再現性は“性質（不変量）”で管理する。
#
# 実務Tips：
#  - MacBook（Apple Silicon）では CUDA は使えないため、Metal(MPS) を優先する。
#  - 推論（補完）だけなら model.eval() ＋ torch.no_grad() でメモリ＆速度最適化。
#  - 学習する場合は DataCollatorForLanguageModeling を用い、mlm_probability を設定する。
#  - トークナイザとモデルは必ず同じモデル名（語彙）を使う（IDずれ防止）。
# =========================================================

import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM


# --- デバイス検出（Mac優先：MPS → CUDA → CPU） ---
def get_best_device() -> torch.device:
    # Apple Silicon + macOS（MPSバックエンド）
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    # 他環境に移植した場合に備えて CUDA も許容
    if torch.cuda.is_available():
        return torch.device("cuda")
    # どれも不可なら CPU
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# --- モデル名（WWM版日本語BERT）：トークナイザとモデルはペアで統一 ---
model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"

# トークナイザ（形態素解析→WordPiece）。特殊トークン：[CLS]/[SEP]/[PAD]/[MASK]/[UNK]
# - `mask_token` と `mask_token_id` は MLM の置換に使用する。
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# MLM 用のヘッド付き BERT（語彙サイズ×隠れ次元の出力層を持つ）
# - 出力：logits 形状 [B, L, |Vocab|]。各位置の語彙分布を返す。
bert_mlm = BertForMaskedLM.from_pretrained(model_name)

# モデルを最適デバイスへ移動（Mac では .cuda() ではなく .to(device) を使う）
bert_mlm = bert_mlm.to(device)

# 推論モード（補完タスクなど）：dropout を停止
bert_mlm.eval()

# --- 参考：MLM 推論の最小例（必要ならコメント解除して確認） ---
# text = f"明日は{tokenizer.mask_token}だ。"
# enc = tokenizer(text, return_tensors='pt')
# enc = {k: v.to(device) for k, v in enc.items()}
# with torch.no_grad():
#     logits = bert_mlm(**enc).logits  # [1, L, V]
# mask_index = (enc["input_ids"][0] == tokenizer.mask_token_id).nonzero(as_tuple=True)[0].item()
# topk = torch.topk(logits[0, mask_index], k=5).indices.tolist()
# candidates = [tokenizer.decode([i]) for i in topk]
# print("Top-5 candidates at [MASK]:", candidates)

# --- 参考：学習時のスケッチ ---
# from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=True,
#     mlm_probability=0.15  # 15% を [MASK]（WWM版モデルの事前学習と整合的に設定）
# )
# # Trainer(...) を用いて学習。PAD位置は自動でロスから除外される（ignore_index=-100）。

[info] using device = mps


Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [3]:
# 5-4
# =========================================================
# 目的：
#  - 入力文に含まれる [MASK] を含めて、日本語BERT用トークナイザでサブワード列へ分割し、挙動を確認する。
#
# 理論メモ（重要ポイント）：
#  - MLM（Masked Language Modeling）では、[MASK] 位置の語彙分布を予測する（学習時は [MASK] 以外の損失を無視）。
#  - `BertJapaneseTokenizer` は「形態素解析 → WordPiece」の二段で分割する。分割境界は辞書・語彙・バージョンに依存。
#  - `tokenize()` は **特殊トークン [CLS]/[SEP] を付けない**“素のサブワード列”を返す。
#  - `[MASK]` は **特殊トークンとして保存**される想定（= 文字列 "[MASK]" がそのままトークン列に現れる）。
#    ※ 安全のため、リテラル "[MASK]" 直書きより `tokenizer.mask_token` を使うのが堅牢。
#  - `encode()` / `tokenizer(..., return_tensors='pt')` は **ID列**を返し、既定で [CLS]/[SEP] を付与して下流モデルに直結できる。
# =========================================================

# 保険：tokenizer 未定義なら初期化（WWM版日本語BERTとペアで統一）
try:
    tokenizer
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# --- 入力文：安全のため tokenizer.mask_token を用いて [MASK] を埋め込む ---
text = f"今日は{tokenizer.mask_token}へ行く。"

# --- サブワード列へ分割（特殊トークン [CLS]/[SEP] は付かない）---
tokens = tokenizer.tokenize(text)
print("# tokens:", tokens)

# --- ID への写像（[MASK] が mask_token_id に対応することを確認）---
ids = tokenizer.convert_tokens_to_ids(tokens)
print("# ids:", ids)

mask_tok = tokenizer.mask_token  # 例："[MASK]"
mask_id = tokenizer.mask_token_id  # 例：mask の語彙ID
mask_pos_tok = tokens.index(mask_tok) if mask_tok in tokens else None
mask_pos_id = ids.index(mask_id) if mask_id in ids else None
print(
    "# mask token:",
    mask_tok,
    "mask id:",
    mask_id,
    "pos(tok):",
    mask_pos_tok,
    "pos(id):",
    mask_pos_id,
)

# --- 参考：特殊トークン付与あり（モデル入力に近い形；[CLS] と [SEP] を含む）---
enc = tokenizer(text, return_tensors="pt", add_special_tokens=True)
print("# input_ids:", enc["input_ids"].tolist())
print("# attention_mask:", enc["attention_mask"].tolist())
print("# tokens (with specials):", tokenizer.convert_ids_to_tokens(enc["input_ids"][0]))

# --- 復元表示（可逆ではない場合がある点に注意：空白や正規化、未知語の影響）---
print(
    "# decode (skip specials):",
    tokenizer.decode(enc["input_ids"][0], skip_special_tokens=True),
)

# tokens: ['今日', 'は', '[MASK]', 'へ', '行く', '。']
# ids: [3246, 9, 4, 118, 3488, 8]
# mask token: [MASK] mask id: 4 pos(tok): 2 pos(id): 2
# input_ids: [[2, 3246, 9, 4, 118, 3488, 8, 3]]
# attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1]]
# tokens (with specials): ['[CLS]', '今日', 'は', '[MASK]', 'へ', '行く', '。', '[SEP]']
# decode (skip specials): 今日 は へ 行く 。


In [4]:
# 5-5
# =========================================================
# 目的：
#  - [MASK] を含む文 `text` を符号化し、MLM（BertForMaskedLM）へ入力して
#    各トークン位置の語彙分布（logits）を得る。
#
# 理論メモ：
#  - MLM（Masked Language Modeling）では [MASK] 位置での語彙分布 p(token | context) を予測する。
#    モデル出力 `logits` は形状 [B, L, |V|]（バッチ×系列長×語彙サイズ）であり、
#    「分類スコア」は“各トークン位置における語彙分類のスコア”を意味する。
#  - `tokenizer.encode(text, return_tensors='pt')` は **ID列テンソル** を返す（既定で [CLS]/[SEP] 付与）。
#    研究・実務では `tokenizer(text, return_tensors='pt', padding=..., truncation=...)` の使用が推奨だが、
#    単文・短文であれば `encode` でも問題ない。
#  - attention_mask を省略した場合、PAD を含まない単文では実害は基本的にない。
#    ただしバッチやパディングを伴う運用では attention_mask を明示するのが安全。
#  - デバイス：MacBook(Apple Silicon) では CUDA は使えない。Metal(MPS) を優先し、
#    それも不可なら CPU を用いる。モデルとテンソルは**常に同じ device** に揃える。
# =========================================================

import torch


# --- デバイス検出（Mac優先：MPS → CUDA → CPU） ---
def get_best_device() -> torch.device:
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")  # Apple GPU (Metal)
    if torch.cuda.is_available():
        return torch.device("cuda")  # 他環境への持ち出し用の保険
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# 文章を符号化し、GPU（またはMPS/CPU）に配置する。
# - encode は既定で special tokens（[CLS]/[SEP]）を付与する。
# - return_tensors='pt' により torch.Tensor（形状 [B=1, L]）で取得。
input_ids = tokenizer.encode(
    text, return_tensors="pt"
)  # 例：tensor([[CLS, ..., MASK, ..., SEP]])
# 元コード： input_ids = input_ids.cuda()
# Mac では .cuda() は不可。device に合わせて移動する：
input_ids = input_ids.to(device)

# 念のためモデル側も同じデバイスへ（5-3でto(device)済みなら実質ノーオペ）
bert_mlm = bert_mlm.to(device)
bert_mlm.eval()  # 推論のみ。学習なら train() に切替

# BERTに入力し、語彙スコア（logits）を得る。
# ※ 元コメントの「系列長を揃える必要がない」は“単文かつ短文で PAD を使わない”場合に限って概ね正しい。
#    バッチや長文運用では padding / attention_mask を明示すること。
with torch.no_grad():  # 推論では計算グラフを作らない：省メモリ・高速化
    output = bert_mlm(
        input_ids=input_ids
    )  # 他に attention_mask 等を省略（単文短文の前提）
    scores = output.logits  # 形状 [B, L, |V|]：各位置の語彙ロジット

# --- 参考（任意）：[MASK] 位置の Top-k 候補を取得する手順（コメントアウト） ---
# mask_id = tokenizer.mask_token_id
# mask_pos = (input_ids[0] == mask_id).nonzero(as_tuple=True)[0].item()  # [MASK] の位置（L次元）
# topk = torch.topk(scores[0, mask_pos], k=5).indices.tolist()
# candidates = [tokenizer.decode([i]) for i in topk]  # 語彙ID → 文字列
# print("Top-5 candidates at [MASK]:", candidates)

# --- 参考（理論補足）---
# - Self-Attention：softmax(QK^T / √d_k) V。系列長 L に対して計算量 O(L^2 H)。
#   長文・バッチ運用では L を適切に管理（truncation/stride/動的パディング）すること。
# - PAD を含む場合は attention_mask=0 の位置が softmax 前に -∞ 相当で抑制され、注意が向かない（勾配も基本流れない）。

[info] using device = mps


In [5]:
# 5-6
# =========================================================
# 目的：
#  - MLM の出力 `scores`（形状 [B, L, |V|]）から [MASK] 位置の最尤トークン（argmax）を選び，
#    元の文の [MASK] をそのトークンで置換して，人間可読な文に復元する。
#
# 理論メモ：
#  - `scores[0, pos]` は語彙分布（ロジット）。argmax は MAP 推定（最尤語彙）に対応する。
#  - [MASK] の語彙IDを **固定値で仮定しない**（モデルによって異なる）。必ず `tokenizer.mask_token_id` を用いる。
#  - サブワード（WordPiece）の場合，`convert_ids_to_tokens` は '##' 接頭辞が付くことがある。
#    文字列復元としては `tokenizer.decode([id])` の方が自然なことが多い（空白処理や正規化を担う）。
#  - 複数 [MASK] がある場合：ここでは「独立同時置換」（各位置で argmax）を行う。
#    逐次的に再エンコードして条件付きで更新する「反復置換」は別戦略（より厳密だが計算増）。
# =========================================================

import torch

# --- 1) [MASK] の語彙IDを取得（モデル依存；固定値を使わない） ---
mask_id = (
    tokenizer.mask_token_id
)  # 例：東北大日本語BERTでは 4 のことが多いが、必ず動的に取得する
mask_tok = tokenizer.mask_token  # 例："[MASK]"

# --- 2) 入力ID列から [MASK] の位置を列挙（複数対応） ---
# input_ids: 形状 [B=1, L] を想定（5-5の encode に一致）
mask_positions = (input_ids[0] == mask_id).nonzero(as_tuple=True)[0].tolist()
if len(mask_positions) == 0:
    raise ValueError(
        "入力系列に [MASK] が見つかりません。text に tokenizer.mask_token を含めてください。"
    )

# --- 3) 各 [MASK] 位置で最尤トークンIDを取得（argmax over vocab） ---
# scores: 形状 [B=1, L, |V|] を想定（5-5の出力 logits）
pred_token_ids = []
for pos in mask_positions:
    # scores[0, pos] : [|V|] ベクトル → 最尤語彙ID
    best_id = scores[0, pos].argmax(-1).item()
    pred_token_ids.append(best_id)

# --- 4) ID → 文字列へ変換（decode を優先） ---
# decode は空白や '##' の扱いをよしなに調整してくれる（単一IDでも有用）
pred_tokens = [
    tokenizer.decode([tid], skip_special_tokens=True).strip() for tid in pred_token_ids
]

# フォールバック（必要時）：WordPieceの『##』を除去して素片文字列だけを得る
pred_tokens_fallback = [
    tokenizer.convert_ids_to_tokens(tid).lstrip("##") for tid in pred_token_ids
]

# decode 側で万が一空文字になった箇所はフォールバックを使う
pred_tokens = [
    pt if pt != "" else pf for pt, pf in zip(pred_tokens, pred_tokens_fallback)
]

# --- 5) 元のテキスト中の [MASK] を順に置換（複数 [MASK] に対応） ---
# str.replace は全置換になるため、分割→挿入→連結で「順番に」置換する
parts = text.split(mask_tok)  # [seg0, seg1, ..., segN]  （N = マスク数）
filled_segments = []
for i, seg in enumerate(parts[:-1]):
    filled_segments.append(seg)
    # i 番目の [MASK] に対する候補を挿入
    filled_segments.append(pred_tokens[i] if i < len(pred_tokens) else "")
filled_segments.append(parts[-1])

text_filled = "".join(filled_segments)
print(text_filled)

# --- 備考 ---
# - 「独立同時置換」では各 [MASK] を互いに独立に推定するため，相互依存（共起制約）を捉えにくい。
#   文脈整合性を高めたい場合は，1つずつ置換→再エンコード→再推論の反復法を検討する。
# - サブワードが選ばれた場合でも decode により自然な表記になりやすいが，
#   場合によっては不自然な連結になる可能性がある（語境界／表記揺れのため）。

今日は、へ行く。


In [6]:
# 5-7
# =========================================================
# 説明のコメント付き：最初の [MASK] を上位K候補で穴埋めする関数
# ---------------------------------------------------------
# ■ 目的
#   - 文章中の「最初の」 [MASK] を、MLM（BertForMaskedLM）の出力ロジットから
#     上位K（num_topk）件の最尤候補で置換した文を生成する。
#   - 戻り値は「穴埋め後テキストのリスト」と「各候補のスコア（ロジット）」。
#
# ■ 理論メモ
#   - MLM（Masked Language Modeling）は、[MASK] 位置で p(token | context) を推定するタスク。
#     モデル出力 logits の形状は [B, L, |Vocab|]。位置 pos の分布は logits[0, pos]。
#   - 最尤候補は argmax、上位K候補は topk（確率化するなら softmax / 温度 / top-p 等も可）。
#   - 特殊トークンの ID（[MASK], [CLS], [SEP] など）は **モデル依存**。ハードコード禁止。
#     → `tokenizer.mask_token_id` / `tokenizer.mask_token` を常に用いる。
#   - サブワード（WordPiece）の '##' 接頭辞を手作業で剥がすより、`tokenizer.decode([id])`
#     を使う方が自然な表記（空白・正規化）になりやすい。
#   - Self-Attention の計算量は O(L^2 H)。本関数は単文・短文想定のため padding/attention_mask 省略でも実害は小さいが、
#     実運用（バッチ/長文）では tokenizer(..., padding/truncation, return_tensors='pt') を推奨。
#
# ■ 実装メモ
#   - デバイスは「モデルの実デバイス」に合わせる（MPS/CUDA/CPU）。Macでは MPS（Metal）を想定。
#   - [MASK] が複数ある場合は **最初の一つだけ** を対象にする（要件どおり）。拡張は容易。
#   - スコア（ロジット）は CPU の NumPy に落として返す（ログや外部I/Oに使いやすい）。
# =========================================================

from typing import List, Tuple


def predict_mask_topk(
    text: str, tokenizer, bert_mlm, num_topk: int
) -> Tuple[List[str], "np.ndarray"]:
    """
    文章中の「最初の」[MASK]を、スコア上位のトークンで置換する。
    上位何位まで使うかは num_topk で指定。
    出力は（穴埋め後テキストのリスト, スコア配列[NumPy]）。
    """
    if num_topk <= 0:
        raise ValueError("num_topk は 1 以上を指定してください。")

    # --- 1) エンコード：既定で [CLS]/[SEP] が付く。return_tensors='pt' で ID列テンソル化（B=1, L）
    enc = tokenizer(
        text, return_tensors="pt"
    )  # 単文・短文のため padding/truncation は省略

    # --- 2) デバイス整合：モデルの実デバイスへ移動（Macなら多くは 'mps'）
    device = next(bert_mlm.parameters()).device
    enc = {k: v.to(device) for k, v in enc.items()}

    # --- 3) 推論：勾配は不要（省メモリ・高速化）
    bert_mlm.eval()
    with torch.no_grad():
        logits = bert_mlm(**enc).logits  # [1, L, |Vocab|]

    input_ids = enc["input_ids"]  # [1, L]
    mask_id = tokenizer.mask_token_id  # モデル依存；固定値を使わない
    mask_tok = tokenizer.mask_token  # 文字列 "[MASK]"

    # --- 4) 「最初の [MASK]」の位置を取得
    mask_positions = (input_ids[0] == mask_id).nonzero(as_tuple=True)[0].tolist()
    if not mask_positions:
        raise ValueError(
            "入力文に [MASK] が見つかりません。text に tokenizer.mask_token を含めてください。"
        )
    pos = mask_positions[0]  # 最初の [MASK]

    # --- 5) 上位K候補を取得（ロジットの topk）
    # logits[0, pos] は語彙サイズ |V| のスコアベクトル
    topk = torch.topk(logits[0, pos], k=num_topk)
    ids_topk = topk.indices.tolist()  # 候補の語彙ID列
    scores_topk = topk.values.detach().to("cpu").numpy()  # NumPy に落として返す

    # --- 6) ID → 文字列へ復元（decode を優先；空になった場合はトークン文字列をフォールバック）
    tokens_topk = []
    for tid in ids_topk:
        s = tokenizer.decode([tid], skip_special_tokens=True).strip()
        if not s:
            s = tokenizer.convert_ids_to_tokens(tid).lstrip("##")
        tokens_topk.append(s)

    # --- 7) 「最初の1箇所のみ」置換：全置換を避けるため split → 1回だけ挿入 → 連結
    pre, sep, post = text.partition(mask_tok)  # 最初の [MASK] で三分割
    if sep == "":
        # 理論的にはここに来ない（上で検出済み）が、防御的に処理
        return [text], scores_topk

    text_topk: List[str] = [pre + tok + post for tok in tokens_topk]

    return text_topk, scores_topk


# --- 使用例 ---
# 例文：最初の [MASK] のみを置換
text = "今日は[MASK]へ行く。"
text_topk, scores_topk = predict_mask_topk(text, tokenizer, bert_mlm, 10)

# 可読表示：上位候補を上から順に（logits は単調変換で確率に相当。softmax するなら別途）
print(*text_topk, sep="\n")
# （必要なら確率化）例：
# import numpy as np
# probs = np.exp(scores_topk - scores_topk.max())  # softmax の安定版の一部
# probs = probs / probs.sum()
# print("probs:", probs)

今日は、へ行く。
今日ははへ行く。
今日はのへ行く。
今日はにへ行く。
今日はでへ行く。
今日は「へ行く。
今日はへへ行く。
今日はをへ行く。
今日はとへ行く。
今日はからへ行く。


In [7]:
# 5-8
# =========================================================
# 説明のコメント付き：貪欲法（Greedy）による逐次穴埋め
# ---------------------------------------------------------
# ■ 目的
#   - [MASK] を含む文章に対し、左から順に 1 箇所ずつ最尤候補（top-1, argmax）で置換していき、
#     すべての [MASK] を埋めた文を返す。
#
# ■ 理論メモ
#   - MLM（Masked Language Modeling）は「各位置のトークン分布 p(token | context)」を返すが、
#     同時に複数の [MASK] を最適化する（joint）わけではない。
#   - 本関数は **貪欲（greedy）**：最左の [MASK] を top-1 で即確定 → 文を更新 → 次の [MASK] … を繰り返す。
#     - 長所：実装・計算が簡単（前向き推論を #MASK 回まわすだけ）
#     - 短所：**順序依存**かつ**局所最適**に陥りやすい（後の候補が前決めに縛られる）
#   - 代替案（必要に応じて検討）：
#     - **ビーム探索**：上位 b 個の候補で分岐し、スコア合算の高い文を保持
#     - **確率的生成**：top-k / top-p（nucleus）サンプリングで多様性を確保
#     - **スパン復元型**モデル（T5 など）：複数トークンの欠落（span corruption）に強い
#
# ■ 実装上の注意
#   - `predict_mask_topk` は 1 箇所の [MASK] を対象にする前提。ここでは「最初の 1 箇所」を top-1 で埋める挙動を利用。
#   - `tokenizer.mask_token` を使って [MASK] 文字列を特定（モデル依存で文字列が異なる可能性に備える）。
#   - ループ回数は **現在のテキスト中の [MASK] 個数**。各反復で文が更新され、次の [MASK] の分布も変わる。
#   - まれに decode の結果が空文字/特殊トークンになる場合があるため、`predict_mask_topk` 側でフォールバック処理済み。
# =========================================================

from typing import Tuple, List


def greedy_prediction(text: str, tokenizer, bert_mlm) -> str:
    """
    [MASK] を含む文章を入力として、左から順に 1 箇所ずつ最尤候補（top-1）で置換し、
    すべての [MASK] を埋めた文章を返す。
    """
    # モデル依存の [MASK] リテラル（例："[MASK]"）
    mask_tok = getattr(tokenizer, "mask_token", "[MASK]")

    # 互換性のため：もし text にハードコード "[MASK]" が含まれ、mask_tok が異なるなら置換
    if mask_tok != "[MASK]" and "[MASK]" in text:
        text = text.replace("[MASK]", mask_tok)

    # 現在の文に残っている [MASK] の数だけ繰り返し
    num_masks = text.count(mask_tok)
    for _ in range(num_masks):
        # 1 箇所だけ（最初の [MASK]）を top-1 で置換
        filled_list, _scores = predict_mask_topk(text, tokenizer, bert_mlm, num_topk=1)
        # predict_mask_topk は常に少なくとも 1 件返す想定（例外時は raise）
        new_text = filled_list[0]

        # 防御的：もし何らかの理由でテキストが変わらなければ早期終了（無限ループ回避）
        if new_text == text:
            break

        text = new_text

    return text


# --- 使用例（2 マスクを貪欲に埋める） ---
text = "今日は[MASK][MASK]へ行く。"
result = greedy_prediction(text, tokenizer, bert_mlm)
print(result)

# 参考：
# - 生成の質を上げたい場合は、greedy の代わりに top-k / top-p サンプリングやビーム探索を検討。
# - 「今日は [MASK] 駅 へ行く」のように右文脈を具体化すると、名詞候補が上がりやすい。

今日は、東京へ行く。


In [8]:
# 5-9
# =========================================================
# 目的：
#   - 5 連続の [MASK] を含む文を、貪欲法（greedy）で左から順に 1 トークンずつ埋めて最終文を得る。
#
# 理論メモ：
#   - BERT の MLM は「各位置の 1 トークン」分布 p(token | 文脈) を返す（自己回帰ではない）。
#   - 貪欲法は「最左の [MASK] を argmax で即確定 → 文を更新 → 次の [MASK] …」の繰り返し。
#     長所：計算が軽い。短所：局所最適に陥りやすく、記号や助詞の“安全牌”が連続しがち。
#   - 5 連続 [MASK] は内容語（名詞句）よりも、高頻度の助詞・記号が上位に来やすい（1 トークン最尤の積み重ね）。
#   - 品質を上げたい場合：ビーム探索／top-k・top-p サンプリング／名詞フィルタ／右文脈強化（例：「…[MASK][MASK]駅へ行く」）が有効。
#
# 前提：
#   - `predict_mask_topk(text, tokenizer, bert_mlm, k)` と `greedy_prediction(text, tokenizer, bert_mlm)` は前セルで定義済み。
#   - `tokenizer` と `bert_mlm` は同一モデル名でロード済み、かつ同一 device（MPS/CUDA/CPU）に配置済み。
# =========================================================

# 入力：5 連続 [MASK]
text = "今日は[MASK][MASK][MASK][MASK][MASK]"

# 実行：貪欲法で順次埋める
result = greedy_prediction(text, tokenizer, bert_mlm)
print(result)


# ---------------------------------------------------------
# （任意）逐次の埋め替え過程を可視化したい場合は、以下のデバッグ関数を使う：
# ---------------------------------------------------------
def greedy_prediction_debug(text, tokenizer, bert_mlm):
    """
    貪欲法の各ステップで文を表示するデバッグ版。
    """
    mask_tok = getattr(tokenizer, "mask_token", "[MASK]")
    # 互換：text が "[MASK]" リテラルの場合、モデル依存の mask_tok に揃える
    if mask_tok != "[MASK]" and "[MASK]" in text:
        text = text.replace("[MASK]", mask_tok)

    step = 0
    while mask_tok in text:
        step += 1
        print(f"# step {step} (before): {text}")
        filled_list, _ = predict_mask_topk(text, tokenizer, bert_mlm, num_topk=1)
        text = filled_list[0]
        print(f"# step {step}  (after): {text}")
    return text


# デバッグ出力を見たいときに有効化：
# _ = greedy_prediction_debug('今日は[MASK][MASK][MASK][MASK][MASK]', tokenizer, bert_mlm)

# ---------------------------------------------------------
# （応用）助詞・記号を簡易フィルタして内容語を優先したい場合の例（ヒューリスティック）
# ---------------------------------------------------------
import re
import torch
from typing import List

# “名詞っぽさ”の非常に粗いヒューリスティック（厳密には形態素解析を推奨）
_PUNCT_OR_PARTICLES = set(
    [
        "、",
        "。",
        "・",
        "「",
        "」",
        "（",
        "）",
        "[",
        "]",
        "(",
        ")",
        "は",
        "の",
        "に",
        "で",
        "を",
        "と",
        "へ",
        "から",
        "より",
        "や",
        "も",
        "が",
    ]
)
_JA_WORD_PATTERN = re.compile(
    r"^[\u3040-\u30ff\u3400-\u9fffA-Za-z0-9ー・]+$"
)  # 仮名・漢字・英数・長音・中黒


def predict_mask_topk_filtered(
    text: str, tokenizer, bert_mlm, num_topk: int, k_expand: int = 50
) -> List[str]:
    """
    predict_mask_topk の派生：top-k を広めに取ってから（k_expand）、助詞・記号を粗く除外し、
    上位 num_topk を返す。完全ではないが内容語が出やすくなることがある。
    """
    enc = tokenizer(text, return_tensors="pt")
    device = next(bert_mlm.parameters()).device
    enc = {k: v.to(device) for k, v in enc.items()}
    bert_mlm.eval()
    with torch.no_grad():
        logits = bert_mlm(**enc).logits
    input_ids = enc["input_ids"]
    mask_id = tokenizer.mask_token_id
    mask_tok = tokenizer.mask_token

    pos_list = (input_ids[0] == mask_id).nonzero(as_tuple=True)[0].tolist()
    if not pos_list:
        return [text]
    pos = pos_list[0]

    # 広めの top-k を取り、その中からフィルタで間引く
    k = max(num_topk, k_expand)
    cand_ids = torch.topk(logits[0, pos], k=k).indices.tolist()

    def _token_str(tid: int) -> str:
        s = tokenizer.decode([tid], skip_special_tokens=True).strip()
        return s if s else tokenizer.convert_ids_to_tokens(tid).lstrip("##")

    cands = []
    for tid in cand_ids:
        s = _token_str(tid)
        # 粗フィルタ：助詞・記号っぽいものを除外
        if (s in _PUNCT_OR_PARTICLES) or (not _JA_WORD_PATTERN.match(s)):
            continue
        cands.append(s)
        if len(cands) >= num_topk:
            break

    if not cands:
        # すべて弾かれたら元の top-1 を返す（安全弁）
        cands = [_token_str(cand_ids[0])]

    pre, sep, post = text.partition(mask_tok)
    return [pre + c + post for c in cands]


# 使い方（任意）：
# text = '今日は[MASK][MASK][MASK][MASK][MASK]'
# t, _ = predict_mask_topk(text, tokenizer, bert_mlm, 1)  # まず 1 個目
# text = t[0]
# text = predict_mask_topk_filtered(text, tokenizer, bert_mlm, num_topk=1)[0]  # フィルタ版で次を埋める …といった併用も可能

今日は社会社会的な地位


In [9]:
# 5-10
# =========================================================
# 理論的な説明コメント（ビームサーチによる連続[MASK]の穴埋め）
# ---------------------------------------------------------
# ■ 目的
#   - 文章中の複数の [MASK] を、ビームサーチ（Beam Search）により高スコアな候補列で逐次埋める。
#   - 各ステップで「最初の [MASK]」を top-k 展開し、合計スコアの高い上位k本の“部分列”を保持して次ステップへ進む。
#
# ■ ビームサーチの理論背景
#   - 貪欲法（greedy）は各ステップで argmax を即確定するため、系列全体の最適性を考慮できず局所最適に陥りやすい。
#   - ビームサーチは、各ステップで上位k候補（ビーム幅）を“並走”させ、累積スコアで比較することで
#     系列全体の尤もらしさを近似的に最大化する探索法である。
#
# ■ スコア設計に関する注意（重要）
#   - 本コードでは predict_mask_topk が返す「scores_topk_inner」を**そのまま加算**している。
#     これが「ロジット（未正規化スコア）」である場合、理論上は各ステップごとに log-softmax で
#     **対数確率**へ変換し、その和（累積 log p）を最大化するのが尤度に整合的である。
#   - ただし、上位候補の相対順位が大きく変わらない前提では「ロジット和」でも経験的に動作することがある。
#     再現性・理論整合性を重視するなら、predict_mask_topk 側で log-softmax へ変換してから返す設計が望ましい。
#
# ■ 探索幅（num_topk）の解釈
#   - 本コードでは num_topk を「各ステップの展開幅（top-k）」と「ビーム幅（保持本数）」の**兼用**として使っている。
#   - 実務では通常、`top_k_each`（展開幅）と `beam_size`（保持幅）を**別パラメータ**に分けると性能/計算の調整がしやすい。
#
# ■ 複数[MASK]の扱い
#   - 各ステップでは「現在の部分解テキスト」の**最初の [MASK] だけ**を predict_mask_topk で埋める。
#   - ステップ数は初期の num_mask だけ繰り返す。毎ステップでビーム内の各文が1つずつ [MASK] を減らすイメージ。
#
# ■ 脱落・同値・冗長性
#   - 同一文字列が複数の経路から生成される場合がある（冗長化）。一意化すると効率が上がる場合もある。
#   - 同スコア同士の順序は argsort 依存（安定ではない可能性）。必要なら安定ソートや副次指標で決める。
#
# ■ 複語復元とモデルの性質
#   - BERT-MLM は「位置ごとの1トークン」予測であるため、連続[MASK]で**名詞句**を復元するのは構造上難しい。
#   - ビームサーチはその制約を緩和するが、抜本的には「スパン復元型（T5系）」の方が親和性が高い。
#
# ■ 計算量の見積り
#   - おおよそ O( num_mask × num_topk × fwd_cost ) で推論を繰り返す（ビーム内の全経路で前向きが走る）。
#   - モデルを eval() にし、no_grad を徹底、系列長（max_length）を適切化することで実時間を抑える。
# =========================================================


def beam_search(text, tokenizer, bert_mlm, num_topk):
    """
    ビームサーチで文章の穴埋めを行う。

    理論注記：
      - num_topk は「各ステップで展開する候補数」かつ「保持するビーム幅」として使っている（兼用）。
      - 合計スコアは単純加算。scores_topk_inner がロジットの場合、
        本来は log-softmax による対数確率和で比較するのが尤度最大化に整合的。
    """
    # 初期の [MASK] 個数（反復回数）。各ステップで最初の1個だけ埋める前提。
    num_mask = text.count("[MASK]")

    # 現時点の上位候補（テキスト列）と、それぞれの累積スコア。
    # 初期状態は「原文（マスクあり）」とスコア0。
    text_topk = [text]
    scores_topk = np.array([0])

    # マスクの数だけステップを繰り返し、各ステップで1つずつ埋める
    for _ in range(num_mask):
        # 現在の各候補テキストごとに、最初の [MASK] を top-k（=num_topk）で展開
        text_candidates = []  # 新たに生成される候補文（展開分すべて）
        score_candidates = []  # それらに対応する「累積スコア」

        for text_mask, score in zip(text_topk, scores_topk):
            # predict_mask_topk は「最初の [MASK]」を top-k 候補で置換した文リストと
            # その“スコア”（ロジット or log確率）ベクトルを返す前提
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            # 生成された各候補文を集約
            text_candidates.extend(text_topk_inner)
            # 既存の累積スコアに、今回のトークン分スコアを加算
            # （理論的には logsoftmax の和が望ましい。ここでは返り値の尺度に依存）
            score_candidates.append(score + scores_topk_inner)

        # step 全体で生まれた（ビーム幅×展開幅）本の候補から、合計スコア上位 num_topk を選抜
        score_candidates = np.hstack(score_candidates)  # 1次元配列へ連結
        idx_list = score_candidates.argsort()[::-1][
            :num_topk
        ]  # 降順上位kのインデックス
        text_topk = [
            text_candidates[idx] for idx in idx_list
        ]  # 次ステップへ残すトップkテキスト
        scores_topk = score_candidates[idx_list]  # 対応スコア

    # すべての [MASK] を埋め切った時点で、上位 num_topk 本の候補文を返す
    return text_topk


text = "今日は[MASK][MASK]へ行く。"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep="\n")

# ---------------------------------------------------------
# 追加の理論的補足と改善余地
# ---------------------------------------------------------
# 1) スコアの正規化：
#    - predict_mask_topk の戻り値がロジットなら、内部で log-softmax に変換し、
#      ここでは「累積 log 確率の和」を加算するのが尤度最大化に一致する。
#
# 2) パラメータ分離：
#    - 「展開幅 top_k_each」と「保持幅 beam_size」を分離し、
#      展開は広く（例：50）、保持は適度（例：5）にする設計が探索/計算のトレードオフ上有利。
#
# 3) 冗長候補の統合：
#    - 同一テキストが複数経路で生成された場合は重複排除（最後に高スコアのみ保持）で効率向上。
#
# 4) 品質向上のための事後フィルタ：
#    - 助詞・記号の除外、品詞ベースのフィルタ（形態素解析）、右文脈の具体化などを併用すると
#      名詞句・固有名詞の自然さが向上しやすい。
#
# 5) モデル選択：
#    - 連続[MASK]の“句”復元に本質的に強いのは、スパン汚染で事前学習した T5 系など。
#      BERT-MLM + ビームでも改善は見込めるが、構造的適性は T5 系に分がある。

今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲーム##センターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。


In [10]:
# 5-11（説明コメント付き：5連続 [MASK] をビームサーチで穴埋めして表示）
# ======================================================================
# 目的：
#  - 連続する5箇所の [MASK] を、前セルで定義したビームサーチ（beam_search）で逐次的に埋め、
#    最終候補の上位10文を改行区切りで表示する実行セル。
#
# 前提（このセルが成立するための条件）：
#  - tokenizer と bert_mlm が同じモデル名（語彙）でロード済みであること。
#  - beam_search 関数と predict_mask_topk 関数が前セルで定義済みであること。
#  - predict_mask_topk は「最初の [MASK] を top-k 候補で置換した文リスト」と
#    「そのときのスコア（通常はロジット。理想は log-softmax）」を返す仕様であること。
#  - ここでは [MASK] の“文字列”を直書きしているため、tokenizer.mask_token が "[MASK]" と一致している前提。
#   （もし異なる場合は、事前に text の [MASK] を tokenizer.mask_token に置換すること。）
#
# 理論メモ（なぜビームサーチを使うか）：
#  - BERT の MLM は各位置の「1トークン分布」を返すため、貪欲法（その場で argmax 確定）だと
#    局所最適（助詞・記号の連鎖）に偏りやすい。
#  - ビームサーチは、各ステップで上位k候補（ビーム）を“並走”させ、累積スコアで比較するため、
#    系列全体の尤もらしさを近似的に最大化できる（greedy より頑健）。
#  - ただし、beam_search の内部で「スコアの足し方」がロジット和のままだと理論的な尤度比較とはずれる。
#    可能なら predict_mask_topk 側で log-softmax に変換し、ここでは「累積対数確率の和」で比較するのが望ましい。
#
# 期待される出力の傾向：
#  - 文脈が「今日は [MASK][MASK][MASK][MASK][MASK]」と抽象的なので、頻度の高い助詞や記号（、・は・の・に…）が
#    上位に出やすいのは仕様どおり（バグではない）。
#  - 右文脈を具体化（例：「今日は [MASK][MASK] 駅 へ行く」）すると名詞が上がりやすい。
#
# 実行コストの概算：
#  - 概ね O( #MASK × ビーム幅 × 前向き計算コスト )。
#  - モデルは eval()・no_grad を徹底し、max_length を過不足なく設定することで時間・メモリを抑制。
#
# 表示：
#  - print(*text_topk, sep='\n') は、ビーム上位10件（文字列リスト）を1行ずつ出力する。
# ======================================================================

text = "今日は[MASK][MASK][MASK][MASK][MASK]"  # 入力文：5連続の [MASK]（span 的な欠落を想定）

# ビームサーチの適用：
# - 第1ステップ：最初の [MASK] を top-10 展開 → 上位10本の部分解を保持
# - 第2〜5ステップ：各部分解で再び「最初の [MASK]」を top-10 展開 → 合計スコア上位10本を保持
# - 結果：全ての [MASK] を埋めた候補文の上位10件が得られる
text_topk = beam_search(text, tokenizer, bert_mlm, 10)

# 出力：上位候補を改行区切りで列挙
print(*text_topk, sep="\n")

今日は社会社会学会所属。
今日は社会社会学会会長。
今日は社会社会に属する。
今日は時代社会に属する。
今日は社会社会学会理事。
今日は時代社会にあたる。
今日は社会社会にある。
今日は社会社会学会会員。
今日は時代社会にある。
今日は社会社会になる。
